<a href="https://colab.research.google.com/github/wheath/thelazyscholar/blob/main/thelazyscholar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install weaviate-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 11.2 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=ea7d66b774674a83fcc820ffc71f320bc40853838aea1421c7699a58df85d99c
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1


In [5]:
import os
import weaviate

client = weaviate.Client(
    url="https://thelazyscholar-uuhw7wj2.weaviate.network",
    additional_headers={
        "X-OpenAI-Api-Key": "NYB",   # Replace w/ your OpenAI Key 
    }
)
client.is_ready()


True

In [33]:
#Clear up the schema, so that we can recreate it

#client.schema.delete_all()
client.schema.get()

# Define the Schema object to use `text-embedding-ada-002` on `title` and `content`, but skip it for `url`
article_schema = {
    "class": "Article",
    "description": "A collection of articles",
    "vectorizer": "text2vec-openai",
    "moduleConfig": {
        "text2vec-openai": {
          "model": "ada",
          "modelVersion": "002",
          "type": "text"
        },
        "generative-openai": {
            "model": "gpt-3.5-turbo"
            
        }
    },
    "properties": [{
        "name": "title",
        "description": "Title of the article",
        "dataType": ["string"]
    },
    {
        "name": "execsummary",
        "description": "executive summary of the article",
        "dataType": ["text"]
    },
    {
        "name": "url",
        "description": "URL to the article",
        "dataType": ["string"],
        "moduleConfig": { "text2vec-openai": { "skip": True } }
    }]
}

# add the Article schema
client.schema.create_class(article_schema)

# get the schema to make sure it worked
client.schema.get()

{'classes': [{'class': 'Article',
   'description': 'A collection of articles',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'generative-openai': {'model': 'gpt-3.5-turbo'},
    'text2vec-openai': {'model': 'ada',
     'modelVersion': '002',
     'type': 'text',
     'vectorizeClassName': True}},
   'properties': [{'dataType': ['text'],
     'description': 'Title of the article',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'title',
     'tokenization': 'whitespace'},
    {'dataType': ['text'],
     'description': 'executive summary of the article',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'execs

In [34]:
### Step 2 - configure Weaviate Batch, with
# - starting batch size of 100
# - dynamically increase/decrease based on performance
# - add timeout retries if something goes wrong

client.batch.configure(
    batch_size=10, 
    dynamic=True,
    timeout_retries=3,
#   callback=None,
)

In [35]:
### Step 3 - import data

print("Importing Articles")



with client.batch as batch:
   

        properties = {
            "title": "The Cathedral and the Bazaar",
            "execsummary": "The Cathedral and the Bazaar: Musings on Linux and Open Source by an Accidental Revolutionary (abbreviated CatB) is an essay, and later a book, by Eric S. Raymond on software engineering methods, based on his observations of the Linux kernel development process and his experiences managing an open source project, fetchmail. It examines the struggle between top-down and bottom-up design. The essay was first presented by the author at the Linux Kongress on May 27, 1997 in Würzburg (Germany) and was published as the second chapter of the same‑titled book in 1999.",
            "url": "https://en.wikipedia.org/wiki/The_Cathedral_and_the_Bazaar"
        }
        
        batch.add_data_object(properties, "Article")
        
print("Importing Articles complete")       

Importing Articles
Importing Articles complete


In [9]:
# Test that all data has loaded – get object count
result = (
    client.query.aggregate("Article")
    .with_fields("meta { count }")
    .do()
)
print("Object count: ", result["data"]["Aggregate"]["Article"], "\n")

Object count:  [{'meta': {'count': 1}}] 



In [12]:
def query_weaviate(query):
    
    nearText = {
        "concepts": [query],
        "distance": 0.7,
    }

    properties = [
        "title", "execsummary", "url",
        "_additional {certainty distance}"
    ]

    result = (
        client.query
        .get("Article", properties)
        .with_near_text(nearText)
        .with_limit(10)
        .do()
    )
    
    # Check for errors
    if ("errors" in result):
        print ("\033[91mYou probably have run out of OpenAI API calls for the current minute – the limit is set at 60 per minute.")
        raise Exception(result["errors"][0]['message'])
    
    return result["data"]["Get"]["Article"]

In [13]:
results = query_weaviate("bazar")
print(results)

[{'_additional': {'certainty': 0.8965392410755157, 'distance': 0.20692152}, 'execsummary': 'The Cathedral and the Bazaar: Musings on Linux and Open Source by an Accidental Revolutionary (abbreviated CatB) is an essay, and later a book, by Eric S. Raymond on software engineering methods, based on his observations of the Linux kernel development process and his experiences managing an open source project, fetchmail. It examines the struggle between top-down and bottom-up design. The essay was first presented by the author at the Linux Kongress on May 27, 1997 in Würzburg (Germany) and was published as the second chapter of the same‑titled book in 1999.', 'title': 'The Cathedral and the Bazaar', 'url': 'https://en.wikipedia.org/wiki/The_Cathedral_and_the_Bazaar'}]


In [16]:
def generativeai_weaviate(query):
    
    nearText = {
        "concepts": [query],
        "distance": 0.7,
    }

    properties = [
        "title", "execsummary", "url",
        "_additional {certainty distance}"
    ]

    prompt = "Summarize as a tweet {execsummary}"

    result = (
        client.query
        .get("Article", properties)
        .with_near_text(nearText)
        .with_generate(single_prompt=prompt)
        .with_limit(10)
        .do()
    )
    
    # Check for errors
    if ("errors" in result):
        print ("\033[91mYou probably have run out of OpenAI API calls for the current minute – the limit is set at 60 per minute.")
        raise Exception(result["errors"][0]['message'])
    
    return result["data"]["Get"]["Article"]

In [36]:
results = generativeai_weaviate("bazar")
print(results)

[{'_additional': {'certainty': 0.8965162038803101, 'distance': 0.20696759, 'generate': {'error': None, 'singleResult': 'CatB by Eric S. Raymond is an essay turned book on software engineering methods based on his observations of Linux kernel development and managing an open source project. It explores the conflict between top-down and bottom-up design. Presented in 1997 and published in 1999. #opensource #linux'}}, 'execsummary': 'The Cathedral and the Bazaar: Musings on Linux and Open Source by an Accidental Revolutionary (abbreviated CatB) is an essay, and later a book, by Eric S. Raymond on software engineering methods, based on his observations of the Linux kernel development process and his experiences managing an open source project, fetchmail. It examines the struggle between top-down and bottom-up design. The essay was first presented by the author at the Linux Kongress on May 27, 1997 in Würzburg (Germany) and was published as the second chapter of the same‑titled book in 1999

In [22]:
!pip install scholarly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
  Created wheel for

In [29]:
from scholarly import scholarly, ProxyGenerator

pg = ProxyGenerator()
success = pg.FreeProxies()
scholarly.use_proxy(pg)


In [37]:
from scholarly import scholarly
import sys

#search_query = scholarly.search_pubs('Perception of physical stability and center of mass of 3D objects')
#scholarly.pprint(next(search_query))
#sys.exit()

# Retrieve the author's data, fill-in, and print
# Get an iterator for the author results
search_query = scholarly.search_author('Steven A Cholewiak')
# Retrieve the first result from the iterator
first_author_result = next(search_query)
#scholarly.pprint(first_author_result)

# Retrieve all the details for the author
author = scholarly.fill(first_author_result )
#scholarly.pprint(author)

# Take a closer look at the first publication
first_publication = author['publications'][0]
first_publication_filled = scholarly.fill(first_publication)
scholarly.pprint(first_publication_filled)

# Print the titles of the author's publications
#publication_titles = [pub['bib']['title'] for pub in author['publications']]
#print(publication_titles)

# Which papers cited that publication?
#citations = [citation['bib']['title'] for citation in scholarly.citedby(first_publication_filled)]
#print(citations)

b"{'author_pub_id': '4bahYMkAAAAJ:u5HHmVD_uO8C',\n 'bib': {'abstract': 'The detectability and discriminability of virtual haptic '\n                     'gratings were analyzed in the frequency domain. '\n                     'Detection (Exp. 1) and discrimination (Exp. 2) '\n                     'thresholds for virtual haptic gratings were estimated '\n                     'using a force-feedback device that simulated sinusoidal '\n                     'and square-wave gratings with spatial periods from 0.2 '\n                     'to 38.4 mm. The detection threshold results indicated '\n                     'that for spatial periods up to 6.4 mm (i.e., spatial '\n                     'frequencies >0.156 cycle/mm), the detectability of '\n                     'square-wave gratings could be predicted quantitatively '\n                     'from the detection thresholds of their corresponding '\n                     'fundamental components. The discrimination experiment '\n             